In [86]:
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity

In [88]:
movies_df = pd.read_csv("cleaned_imdb_movies.csv")
tfidf_vectorizer = joblib.load("models/tfidf_vectorizer.pkl")
nearest_neighbors_indices = joblib.load("models/nearest_neighbors_indices.pkl")
svd_model = joblib.load("models/svd_model.pkl")

In [90]:
def get_similar_movies(movie_title):
    match = movies_df[movies_df["primaryTitle"].str.lower() == movie_title.lower()]
    if match.empty:
        print(f"Movie '{movie_title}' not found in dataset.")
        return pd.DataFrame()
    index = match.index[0]
    similar_indices = nearest_neighbors_indices[index][1:]
    return movies_df.iloc[similar_indices].copy()

In [92]:
def predict_rating_for_movie(movie_name, user_id=999):
    try:
        return round(svd_model.predict(user_id, movie_name).est, 2)
    except:
        return None

In [94]:
def test_recommendation(movie_title):
    print(f"\nTest recommendations for: {movie_title}\n")
    similar_movies = get_similar_movies(movie_title)
    if similar_movies.empty:
        print("No similar movies found.")
        return

    similar_movies["PredictedRating"] = similar_movies["primaryTitle"].apply(lambda title: predict_rating_for_movie(title))

    user_genre_vector = tfidf_vectorizer.transform([""])
    genre_scores = cosine_similarity(user_genre_vector, tfidf_vectorizer.transform(similar_movies["genres"])).flatten()
    similar_movies["user_genre_similarity"] = genre_scores

    similar_movies["score"] = (
        0.6 * similar_movies["user_genre_similarity"] +
        0.4 * similar_movies["averageRating"].fillna(0)
    )

    top_movies = similar_movies.sort_values(by="score", ascending=False).head(10)
    print(top_movies[["primaryTitle", "genres", "averageRating", "PredictedRating", "score"]].to_string(index=False))

In [96]:
test_recommendation("Inception")


Test recommendations for: Inception

                             primaryTitle                     genres  averageRating  PredictedRating  score
Three Billboards Outside Ebbing, Missouri         Comedy,Crime,Drama            8.1             6.04   3.24
                        The King's Speech    Biography,Drama,History            8.0             6.11   3.20
                               La La Land         Comedy,Drama,Music            8.0             6.04   3.20
                              Toy Story 2 Adventure,Animation,Comedy            7.9             5.90   3.16
                                  Skyfall  Action,Adventure,Thriller            7.8             6.04   3.12
             Kingsman: The Secret Service    Action,Adventure,Comedy            7.7             6.08   3.08
                               Changeling      Biography,Crime,Drama            7.7             6.12   3.08
                           Kung Fu Hustle      Action,Comedy,Fantasy            7.7             6.

In [98]:
test_recommendation("The Godfather")


Test recommendations for: The Godfather

                                     primaryTitle                  genres  averageRating  PredictedRating  score
                                    The Godfather             Crime,Drama            9.2             5.97   3.68
The Lord of the Rings: The Fellowship of the Ring Adventure,Drama,Fantasy            8.9             5.97   3.56
                                            Se7en     Crime,Drama,Mystery            8.6             6.07   3.44
                                            Queen  Adventure,Comedy,Drama            8.1             6.08   3.24
                                Blade Runner 2049    Action,Drama,Mystery            8.0             6.22   3.20
                                    Weekend Party  Crime,Romance,Thriller            7.9             6.11   3.16
                                 Sound of Freedom   Biography,Crime,Drama            7.6             6.11   3.04
                                    Thalli Manasu     

In [100]:
test_recommendation("The Matrix")


Test recommendations for: The Matrix

                         primaryTitle                    genres  averageRating  PredictedRating  score
                            Guerrilla         Drama,History,War            8.1             6.10   3.24
                          Chowka Bara                   Romance            7.8             5.94   3.12
Harry Potter and the Sorcerer's Stone  Adventure,Family,Fantasy            7.7             6.11   3.08
                   X-Men: First Class             Action,Sci-Fi            7.7             5.95   3.08
                              Madaari            Drama,Thriller            7.6             6.05   3.04
                Taxi to the Dark Side Crime,Documentary,History            7.5             5.96   3.00
                  Legion of the Black                     Music            7.3             6.16   2.92
                   Olmo & the Seagull Documentary,Drama,Romance            7.2             5.92   2.88
                 A Year Ago in Win